In [1]:
# Using the company facts API, we can get a ton of data about many companies
# But tags aren't always the same across companies, and can vary a lot
# Let's analyze this

In [2]:
import scraper

In [ ]:
companies = scraper.get_companies()

In [ ]:
facts = companies[0].get_facts() # example sets of facts for the first company

In [ ]:
# facts["facts"] has keys representing different taxonomies
# e.g.: dei, us-gaap, invest and srt. 

# facts["facts"]["<taxonomy>"]["<tag name>"]["units"] contains keys for each kind of unit
# This is commonly just 'USD'

# facts["facts"]["taxonomy"]["<tag name>"]["units"]["<unit name>"] contains a list of the tag details
# Each is a dict with end, val, accn, fy (fiscal year), fp (quarter), form (e.g. 10-Q), filed (the date)

facts["facts"]["us-gaap"]["Assets"]["units"]["USD"]

# TODO: investigate the different taxonomies, do you want their info?
# Then process the data (e.g. into a dataframe)
    # What exactly is the end date and filed date?
    # Remove duplicate logs of the same value, only log when it changed and merge dates
# Then compare across companies (what tags are shared across companies a lot?)

[{'end': '2009-01-25',
  'val': 3350727000,
  'accn': '0001045810-09-000024',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-08-20'},
 {'end': '2009-01-25',
  'val': 3350727000,
  'accn': '0001045810-09-000036',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-11-19'},
 {'end': '2009-01-25',
  'val': 3350727000,
  'accn': '0001045810-10-000006',
  'fy': 2010,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2010-03-18',
  'frame': 'CY2008Q4I'},
 {'end': '2009-07-26',
  'val': 3298409000,
  'accn': '0001045810-09-000024',
  'fy': 2009,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2009-08-20',
  'frame': 'CY2009Q2I'},
 {'end': '2009-10-25',
  'val': 3463574000,
  'accn': '0001045810-09-000036',
  'fy': 2009,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2009-11-19',
  'frame': 'CY2009Q3I'},
 {'end': '2010-01-31',
  'val': 3585918000,
  'accn': '0001045810-10-000006',
  'fy': 2010,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2010-03-18'},
 {'end': '2010-01-31',
  'va

In [18]:
facts["facts"].keys()

dict_keys(['dei', 'invest', 'us-gaap', 'srt'])